In [1]:
import os
import csv

In [2]:
def addrFile(tweeter,suffix):
    path=addrRoot+str(tweeter)+'/'
    if os.path.exists(path) is False:
         os.makedirs(path)
    address=path+tweeter+suffix
    return address  

In [4]:
#文件保存地址，改为你存放csv文件的完整地址
addrRoot='C:/Users/cascara/Desktop/seedcup/csv/blog/single/'

#是否调试
isDebug=False

#100万+显示
infinity=1000000

In [5]:
tweeter=input('输入博文代号：')#武汉肺炎-搜狐新闻

输入博主姓名： 武汉肺炎-搜狐新闻


In [6]:
#开始
startCollect=True


batch=0
dataDict={}
Title=[]
while(1):
    titleEixst=True
    address=addrFile(tweeter,'batch'+str(batch)+'.csv')    
    print(address)
    
    if os.path.exists(address) is True:
        fp = open(address,'r',newline='',encoding='utf-16')      
        reader=csv.reader(fp)
        for line in reader:
            if titleEixst is True:
                #将标题制成各个
                if startCollect is True:
                    for item in line:
                        #print(item[-1])
                        if isDebug is True:
                            print(item)

                            
                        dataDict[item]=[]
                    startCollect=False
                Title=line    
                if isDebug is True:
                    print(line)
                titleEixst=False
            else:
                for no in range(len(line)):               
                    if(Title[no].find('count')>=0):
                        if line[no].find('0万') > 0:
                            dataDict.get(Title[no]).append(infinity)
                        else:
                            dataDict.get(Title[no]).append(eval(line[no]))
                        
                        #dataDict.get(Title[no]).append((line[no]))
                        
                    else:
                        dataDict.get(Title[no]).append((line[no]))
                    if isDebug is True:
                        print(Title[no])
                        print(line[no])               
                
    
    else:
        print('不存在'+address)
        break
    fp.close()
    batch+=1
#print(dataDict)

C:/Users/cascara/Desktop/seedcup/csv/blog/single/武汉肺炎-搜狐新闻/武汉肺炎-搜狐新闻batch0.csv
C:/Users/cascara/Desktop/seedcup/csv/blog/single/武汉肺炎-搜狐新闻/武汉肺炎-搜狐新闻batch1.csv
C:/Users/cascara/Desktop/seedcup/csv/blog/single/武汉肺炎-搜狐新闻/武汉肺炎-搜狐新闻batch2.csv
C:/Users/cascara/Desktop/seedcup/csv/blog/single/武汉肺炎-搜狐新闻/武汉肺炎-搜狐新闻batch3.csv
不存在C:/Users/cascara/Desktop/seedcup/csv/blog/single/武汉肺炎-搜狐新闻/武汉肺炎-搜狐新闻batch3.csv


In [7]:
print(Title)

['转发created_at', '转发text', '转发reposts_count', '转发comments_count', '转发attitudes_count', '转发id', '转发screen_name', '转发description', '转发gender', '转发followers_count', '转发follow_count']


### 读入数据

In [8]:
originData={}
retweetData={}

labels=[0 for i in range(4)]
cnLabels={}

labels[1]='reposts_count'
labels[2]='comments_count'
labels[3]='attitudes_count'
labels[0]='follow_count'

cnLabels[labels[1]]='转发量'
cnLabels[labels[2]]='评论量'
cnLabels[labels[3]]='点赞量'
cnLabels[labels[0]]='粉丝量'


for i in range(4):
    #print(choice+label)
    try:
        #originData[cnLabels[i]]=np.asarray(dataDict.get('原文'+labels[i])[::-1])
        retweetData[labels[i]]=np.asarray(dataDict.get('转发'+labels[i])[::-1])
    except:
        pass
    try:
        originData[labels[i]]=np.asarray(dataDict.get('原文'+labels[i])[::-1])
        #retweetData[cnLabels[i]]=np.asarray(dataDict.get('转发'+labels[i])[::-1])
    except:
        pass



In [10]:
import re
#工具类，用来去除爬取的正文中一些不需要的链接、标签等
class Tool:
    deleteImg = re.compile('<img.*?>')
    newLine =re.compile('<tr>|<div>|</tr>|</div>')
    
    deleteAite = re.compile('//.*?:')
    
    repostEN=re.compile('//<a.*?>@(.*?)</a>:')#英文字符冒号
    repostCN=re.compile('//<a.*?>@(.*?)</a>：')#中文字符冒号
    
    
    deleteAddr = re.compile('<a.*?>.*?</a>|<a href='+'\'https:')
    deleteTag = re.compile('<.*?>')
    deleteWord = re.compile('回复@|回覆@|回覆|回复')
 
    @classmethod
    def findSource(cls,x):
        sourceName=''
        xEN=xCN=''
        
        xEN = re.findall(cls.repostEN,x)
        xCN = re.findall(cls.repostCN,x)
        
        
        #如果其中一者存在，另一者不存在，即返回该者
        if(len(xCN)==0 and len(xEN)>0):
            sourceName=xEN[0]
            #print(xEN[0])
        elif(len(xEN)==0 and len(xCN)>0):
            sourceName=xCN[0]
            #print(xCN[0]) 
        #若二者都存在，则返回第一位置字符串较小的
        elif(len(xEN)>0 and len(xCN)>0):
            #print(xCN[0])  if(len(xEN[0])>len(xCN[0])) else print(xEN[0])
            sourceName=xCN[0]  if(len(xEN[0])>len(xCN[0])) else xEN[0]

                
        return sourceName

In [11]:
class Categories:
    
    def __init__(self):
        self.compose={}
        #pass
    def add(self,name):
        self.compose[name]={}
        category=self.compose[name]
        category['value']=0 #记录被转次数
        category['target']={}
        category['source']={}
        #category['isS']=0
    def nameExist(self,name):
        if self.compose.get(name) is None:   
            return False
        else:
            return True
        
    def addTarget(self,sourceName,targetName):
        if self.nameExist(sourceName) is False:
            self.add(sourceName)
        if self.nameExist(targetName) is False:
            self.add(targetName)
        
        ##防止循环调用        
        if sourceName == targetName:
            #print(sourceName)
            return 
        if self.compose[targetName]['source'].get(sourceName) is not None:
            #print(sourceName)
            self.compose[targetName]['source'].pop(sourceName)
            
            
        if self.compose[targetName]['source'].get(sourceName) is None:
            self.compose[targetName]['source'][sourceName]=1
        else:
            self.compose[targetName]['source'][sourceName]+=1    
            
        
        if self.compose[sourceName]['target'].get(sourceName) is None:
            self.compose[sourceName]['target'][targetName]=1
        else:
            self.compose[sourceName]['target'][targetName]+=1
        self.compose[sourceName]['value']+=1    
            
    def countAll(self,name):
        '''count =0
        if self.nameExist(name) is False:
            self.add(name)            
            count= 0
        else:'''
        
        targets=self.compose[name]['target']
        if targets == {}:
            self.compose[name]['value']=0
        else:
            for targetName in targets:
                if self.compose[targetName]['target']=={}:
                    self.compose[targetName]['value']=0
                    #self.compose[name]['value']+=1
                else:
                    self.countAll(targetName)
                    self.compose[name]['value']+=self.compose[targetName]['value']
     #数据缺失补充 假定为转发原博主    
    def fillSource(self,tweeter):
        for item in self.compose:
            source=self.compose[item].get('source')
            if  (len(source))!=1 and item !=tweeter:
                self.addTarget(tweeter,item)
            #print(self.compose[item])
                    
            
        

In [12]:
choice='转发'#'原文'#
categories=Categories()
txt=''



for name,text in  zip(dataDict[choice+'screen_name'],dataDict[choice+'text']):
    
    if categories.nameExist(name) is False:
        categories.add(name)
            
            
    sourceName=(Tool.findSource(text))
    if sourceName is not '':    
        categories.addTarget(sourceName,name)
        
    else:
        categories.addTarget(tweeter,name)

categories.countAll(tweeter)#zwysgs')#处理所有，统计所有转发来源量        
categories.fillSource(tweeter)         
            
         

In [13]:
nodes=[]
arcs=[]

category=[]



for i in  categories.compose:
    value=categories.compose[i]['value']
    try:
        source=list(categories.compose[i]['source'])[0]
    except:
        source=tweeter
        #print(i)
    node={  "name":i,
            "symbolSize": 5,
            "draggable": "False",
            "value": value,
            "category": source
         }
            
           
    if value > 0:
        if i==tweeter:
            node["category"]=i
        #change node
        if source !=tweeter:
            #print(i)
            node["category"]=i
        symbolSize=value//10 
        if symbolSize>5:
            node['symbolSize']=symbolSize
        node['label']={
                "normal": {
                    "show": "True"
                }
            }
    
        
        
        #add arcs
        targets=categories.compose[i]['target']
        if targets != {}:
            for target in targets:
                arcs.append({'source':i,'target':target})
        
        
        #归入category
        category.append({'name':i})
    nodes.append(node)
        
        
    #try:
        #categories.compose[i]['target'][i] 
        
        #print(i)
        #print( categories.compose[i])
    #except:
        #pass
#print(nodes)
#print(arcs)
#print(category)

content='【独家｜武汉红十字会：无管理费收入 全靠财政拨款；12名员工人均“工资福利”23万】资料显示，武汉市红十字会为财政拨款单位，几乎全部收入来自财政拨款。2018年武汉红会“人员经费”支出为329.67万元。其中，工资福利279.34万元，其官方公布当年在职人员12人。以此计算，2018年武汉红会人均“人员经费”27.47万元，人均“工资福利”23.28万元。另据其2019年预算数据计算，武汉红会人均领取“工资福利”有望达到27.98万元。而2018年武汉市城镇单位就业人员平均工资为73671元，城镇非私营单位就业人员平均工资为85885元。\
自武汉新冠肺炎疫情爆发以来，武汉红十字会便开始接受全国各地、各界捐赠物资。截至1月29日24:00，武汉红会6天内累计收到社会捐款4.9亿元。 “1月23号以来，11个工作人员已经全部取消年假，还另外招募了近50位志愿者，24小时轮流加班。” 武汉市红十字会在官网上讲述了自己繁忙的工作状态。但与此同时，对于捐款和物资的质疑也接连而至。\
1月25日，有网友称“第一批医疗物资300套防护服到达武汉时，武汉红会要求捐赠者支付6%-8%的服务费，不然就拒收”。武汉市红十字会于两天后辟谣称，其至今为止的所有工作流程中，均不存在任何收费现象。\
1月29日，网上又开始流传“山东寿光无偿援助武汉350吨蔬菜，却被经过武汉相关部门通过超市售卖后，款项流向武汉红会”。对此，武汉红十字会再次发布申明否认称，其从未接收任何单位、任何个人捐赠的“寿光蔬菜”，更没有参与该批蔬菜的分配、售卖，也没有收到过与此相关的任何现金捐赠。'
mid='4467059339091135'
tweeter='武汉肺炎-搜狐新闻'

In [14]:
testData=[]
testData.append(nodes)
testData.append(arcs)
testData.append(category)
testData.append(content)
testData.append(mid)
testData.append(tweeter)


In [15]:
import json

testFile=addrFile(tweeter,'.json')
with open(testFile,'w',encoding='utf-8') as file_obj:
    json.dump(testData,file_obj)
file_obj.close()

In [17]:
from pyecharts import options as opts
from pyecharts.charts import Geo, Page, Bar ,Graph
from pyecharts.faker import Collector, Faker
from pyecharts.globals import CurrentConfig,NotebookType#ChartType, SymbolType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

C = Collector()

@C.funcs
def graph_weibo() -> Graph:
    with open(testFile, "r", encoding="utf-8") as f:
        j = json.load(f)
        nodes, links, categories, cont, mid, userl = j
    c = (
        Graph()
        .add(
            "肺炎",
            nodes,
            links,
            categories,
            repulsion=50,
            linestyle_opts=opts.LineStyleOpts(curve=0.2),
            label_opts=opts.LabelOpts(is_show=False),#True),#
        )
        .set_global_opts(
            legend_opts=opts.LegendOpts(is_show=False),#True),#
            title_opts=opts.TitleOpts(title="武汉肺炎——搜狐新闻微博转发关系图"),
        )
    )
    print(userl)
    return c

Page().add(*[fn() for fn, _ in C.charts]).render()

武汉肺炎-搜狐新闻
武汉肺炎-搜狐新闻


'C:\\Users\\cascara\\learngit\\gitskills\\singleWeibo\\render.html'